# 2nd Patch of Queries

Recall from the first patch of queries: The last query uses a SELECT with multiple tables. Instead of that, write it with an explicit JOIN.

The queries of the 2nd patch:

(1) For each issue, the time of the first and the last event in changes (clue: you can use GROUP BY issue_id plust MAX() and MIN() on the relevant dates).

(2) For each issue, the difference between the first and last event in changes.

(3) For each patchset in each issue, the difference between the first and the last event for that patchset.

(4) For each patchset in each issue, the time for "waiting for review" (as accurate as possible, do assumptions if you need).

(5) Fo reach patchset in each issue, the time for "waiting for developer", similar to (4)

(1) and (2), you can start using changes.issue_id as the id for the issue, but then try to JOIN to issues and get the real Gerrit id.

(3), (4) and (5), you can as well start using changes.issue_id, and later JOIN with issues to get the real Gerrit id.


In [1]:
import MySQLdb

db = MySQLdb.connect(user="root",
                    passwd="",
                    host="localhost",
                    port=3306,
                    use_unicode=True)

cursor = db.cursor()
cursor.execute("SELECT * FROM reviews.changes LIMIT 20") ## test query

20L

From the first patch of queries - The last querry query without JOIN - 
How many patches per issue with the gerrit id of the issue 

In [4]:
cursor.execute("""SELECT COUNT(c.new_value), i.issue, c.issue_id
FROM reviews.changes c, reviews.issues i
WHERE c.issue_id=i.id
GROUP BY c.issue_id
LIMIT 20""")

20L

From the first patch of queries - The last querry query with JOIN - 
How many patches per issue with the gerrit id of the issue with JOIN

In [5]:
cursor.execute("""SELECT COUNT(reviews.changes.new_value), reviews.issues.issue, reviews.changes.issue_id
FROM reviews.changes
INNER JOIN reviews.issues
ON reviews.changes.issue_id=reviews.issues.id
GROUP BY reviews.changes.issue_id
LIMIT 20""")

20L

From the second patch of queries

(1) For each issue, the time of the first and the last event in changes (clue: you can use GROUP BY issue_id plust MAX() and MIN() on the relevant dates).

In [17]:
cursor.execute("""SELECT issue_id, MIN(changed_on),MAX(changed_on)
FROM reviews.changes
GROUP BY issue_id
LIMIT 20""")

20L

(1) query nr 1 + using JOIN to get the Gerrit id

In [19]:
cursor.execute("""SELECT c.issue_id, i.issue, MIN(c.changed_on),MAX(c.changed_on)
FROM reviews.changes c
INNER JOIN reviews.issues i
ON c.issue_id = i.id
GROUP BY issue_id
LIMIT 20""")

20L

(2) For each issue, the difference between the first and last event in changes.

In [20]:
cursor.execute("""SELECT  issue_id, ABS(TIMEDIFF(MAX(changed_on),MIN(changed_on)))
FROM reviews.changes
GROUP BY issue_id
LIMIT 20""")

/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '4012:44:56'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '4063:21:51'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '5498:43:03'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '2850:10:01'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '2946:46:50'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '3067:38:27'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:4: Warning: Truncated incorrect time value: '3198:36:57'


20L

* The warning probably due to some configuration in python... will have to check...

(2) query nr 2 + using JOIN to get the Gerrit id

In [22]:
cursor.execute("""SELECT  c.issue_id, i.issue, ABS(TIMEDIFF(MAX(c.changed_on),MIN(c.changed_on)))
FROM reviews.changes c
INNER JOIN reviews.issues i
ON c.issue_id = i.id
GROUP BY c.issue_id
LIMIT 20""")

/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '4012:44:56'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '4063:21:51'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '5498:43:03'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '2850:10:01'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '2946:46:50'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '3067:38:27'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:6: Warning: Truncated incorrect time value: '3198:36:57'


20L

(3) For each patchset in each issue, the difference between the first and the last event for that patchset.

In [3]:
cursor.execute("""SELECT issue_id, old_value, ABS(TIMEDIFF(MAX(changed_on),MIN(changed_on)))
FROM reviews.changes
WHERE old_value!="" AND issue_id=1
GROUP BY issue_id, old_value
LIMIT 20""")

20L

(3) query nr 3 + using JOIN to get the Gerrit id

In [4]:
cursor.execute("""SELECT c.issue_id, i.issue, c.old_value, ABS(TIMEDIFF(MAX(c.changed_on),MIN(c.changed_on)))
FROM reviews.changes c
INNER JOIN reviews.issues i
ON c.issue_id = i.id
WHERE c.old_value!="" AND c.issue_id=1
GROUP BY c.issue_id, c.old_value
LIMIT 20""")

20L

(4) For each patchset in each issue, the time for "waiting for review" (as accurate as possible, do assumptions if you need).

In [10]:
### create a view of the table changes that contains just the entries where the value of the atribute 
###'field' is 'Code-Review'. This will help to calculate the difference between each time a'Code-Review'
### happens in a patchset and the first upload for that patchset (that we can match in the changes table) 
cursor.execute("""CREATE VIEW reviews.codeReview AS
SELECT * 
FROM reviews.changes 
WHERE field='Code-Review'""")

0L

In [11]:
# the query for matching the right rows to calculate the difference
cursor.execute("""SELECT cr.issue_id, cr.field, cr.old_value, cr.changed_on, cs.field, cs.changed_on, ABS(TIMEDIFF(cr.changed_on,cs.changed_on))
FROM reviews.codeReview cr  
INNER JOIN reviews.changes cs
ON (cr.issue_id=cs.issue_id AND cr.old_value=cs.old_value)
WHERE (cs.field='Upload' AND cs.new_value="")
ORDER BY cr.issue_id
LIMIT 20""")

20L

NOTE: the view still exists - will use it for the next queries

(4) query nr 4 + using JOIN to get the Gerrit id

In [12]:
cursor.execute("""SELECT cr.issue_id, ri.issue, cr.field, cr.old_value, cr.changed_on, cs.field, cs.changed_on, ABS(TIMEDIFF(cr.changed_on,cs.changed_on))
FROM reviews.codeReview cr
INNER JOIN reviews.changes cs
ON (cr.issue_id=cs.issue_id AND cr.old_value=cs.old_value) 
JOIN reviews.issues ri
ON cr.issue_id = ri.id
WHERE (cs.field='Upload' AND cs.new_value="")
ORDER BY cr.issue_id
LIMIT 20""")

20L

(5) For each patchset in each issue, the time for "developers to review" (as accurate as possible, do assumptions if you need). (NOTE: the view still exists)

In [13]:
cursor.execute("""SELECT cr.issue_id, cr.field, cr.old_value, cr.new_value, cr.changed_on, cs.issue_id, cs.field, cs.old_value, cs.new_value, cs.changed_on, ABS(TIMEDIFF(cr.changed_on,cs.changed_on))
FROM reviews.codeReview cr  
INNER JOIN reviews.changes cs
ON (cr.issue_id=cs.issue_id)
WHERE (cr.new_value=-2 AND (cs.field='Upload' AND cs.new_value="" AND cs.old_value=cr.old_value+1))
ORDER BY cr.issue_id
LIMIT 20""")

/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:7: Warning: Truncated incorrect time value: '-1051:15:31'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:7: Warning: Truncated incorrect time value: '-2154:54:12'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:7: Warning: Truncated incorrect time value: '-1051:17:06'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:7: Warning: Truncated incorrect time value: '-2154:54:02'


20L

(5) query nr 5 + using JOIN to get the Gerrit id

In [14]:
cursor.execute("""SELECT cr.issue_id, ri.issue, cr.field, cr.old_value, cr.new_value, cr.changed_on, cs.issue_id, cs.field, cs.old_value, cs.new_value, cs.changed_on, ABS(TIMEDIFF(cr.changed_on,cs.changed_on))
FROM reviews.codeReview cr  
INNER JOIN reviews.changes cs
ON (cr.issue_id=cs.issue_id) 
JOIN reviews.issues ri
ON cr.issue_id = ri.id
WHERE (cr.new_value=-2 AND (cs.field='Upload' AND cs.new_value="" AND cs.old_value=cr.old_value+1))
ORDER BY cr.issue_id
LIMIT 20""")

/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:9: Warning: Truncated incorrect time value: '-1051:15:31'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:9: Warning: Truncated incorrect time value: '-2154:54:12'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:9: Warning: Truncated incorrect time value: '-1051:17:06'
/home/seneca/V-ENV/ipython-S/lib/python2.7/site-packages/ipykernel/__main__.py:9: Warning: Truncated incorrect time value: '-2154:54:02'


20L